In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from wordcloud import WordCloud

In [140]:
cols= ['dataset','text', 'logical_fallacies', 'source']
df = pd.DataFrame(columns= cols)

In [141]:
df_1 = pd.read_csv('../data/cleaned/1_logicClimate.csv')
df_2 = pd.read_csv('../data/cleaned/2_Huggingface_dataset.csv', index_col=0)
df_3 = pd.read_csv('../data/cleaned/3_CoCoLoFa_merged.csv')
df_4 = pd.read_csv('../data/cleaned/4_falcon_df_multilabel_annotations.csv')
df_5_1 = pd.read_csv('../data/cleaned/5_climate.csv')
df_5_2 = pd.read_csv('../data/cleaned/5_covid.csv')
df_6 = pd.read_csv('../data/cleaned/6_MAFALDA.csv')
# df_7 = pd.read_csv('../data/cleaned/7_Reddit_dataset.csv', index_col=0) #identical to 4
df_8 = pd.read_csv('../data/cleaned/8_argotario.csv')
df_9 = pd.read_csv('../data/cleaned/9_Goffredo_used_in_MAFALDA_cleaned.csv')

### 1

In [ ]:

df_1['dataset']=1
df_1 = df_1.rename(columns={ 'original_url': 'source', 'source_article': 'text'})
df_1 = df_1.loc[:,cols ]
df_1 = df_1[df_1['logical_fallacies']!='fallacy of logic']
df_1.head()

### 2

In [ ]:

df_2['dataset']=2
df_2 = df_2.rename(columns={ 'statement': 'text', 'label': 'logical_fallacies'})
df_2 = df_2.loc[:,['dataset', 'text', 'logical_fallacies']]
df_2 = df_2[df_2['logical_fallacies']!='fallacy of logic']
df_2.head()

### 3

In [ ]:
df_3 = df_3[['fallacy', 'comment']]
df_3 = df_3.rename(columns={'fallacy':'logical_fallacies', 'comment': 'text' })
df_3['dataset'] = 3
df_3 = df_3.loc[:,['dataset','text', 'logical_fallacies']]
df_3.head()

### 4

In [ ]:

df_4 = df_4[['fallacy','main_tweet']]
df_4 = df_4.rename(columns={'fallacy':'logical_fallacies','main_tweet': 'text' })
df_4.head()


In [146]:
i = 0

df_splitted_fallacies = pd.DataFrame(columns=['logical_fallacies', 'text'])
rows_to_delete=[]

for i in range(len(df_4)):
    # print(i)
    current_fallacies = df_4.iloc[i, 0]
    # print('current_fallacies:', current_fallacies)
    current_fallacies = current_fallacies.replace("'", "").replace('[', '').replace(']','')
    splitted_fallacies = current_fallacies.split(sep=',')
    if len(splitted_fallacies)==1:
        df_4.iloc[i, 0] = current_fallacies
    if len(splitted_fallacies)>1:
        for index, f in enumerate(splitted_fallacies):
            new_row = {'logical_fallacies':f, 'text': df_4.iloc[i, 1]}
            df_splitted_fallacies = pd.concat([df_splitted_fallacies, pd.DataFrame([new_row])])
            rows_to_delete.append(i)

    
    

In [147]:
 #remove duplicates
rows_to_delete = list(set(rows_to_delete))

#drop rows with multilabels
df_4 = df_4.drop(axis=0, index=rows_to_delete)

In [ ]:
# concate new rows to dataframe
df_4 = pd.concat([df_4, df_splitted_fallacies])
df_4.head()

In [149]:
df_4['dataset']=4
df_4 = df_4.loc[:,['dataset', 'text', 'logical_fallacies']]

### 5

In [ ]:
df_5_1.head()

In [ ]:
df_5_1 = df_5_1[['fact_checked_segment', 'logical_fallacies', 'article']]
df_5_1 = df_5_1.rename(columns={'fact_checked_segment': 'text', 'article': 'source'})
df_5_1['dataset'] = 5
df_5_1 = df_5_1.loc[:,['dataset', 'text', 'logical_fallacies', 'source']]
df_5_1.head()

In [ ]:
df_5_2.head()

df_5_2 = df_5_2[['claim', 'logical_fallacies']]
df_5_2 = df_5_2.rename(columns={'claim': 'text' })
df_5_2['dataset'] = 5
df_5_2 = df_5_2.loc[:,['dataset', 'text', 'logical_fallacies']]
df_5_2.head()


### 6

In [ ]:
df_6.head()
df_6 = df_6.rename(columns={'text_only': 'text', 'labels_only': 'logical_fallacies'})
df_6['dataset'] = 6
df_6 = df_6.loc[:,['dataset', 'text', 'logical_fallacies']]
df_6.head()

### 8

In [ ]:
pd.set_option('display.max_colwidth', None)
df_8.tail(10)

In [ ]:
df_8.head()
df_8 = df_8.dropna()
df_8 = df_8.rename(columns={'claim': 'text' })
df_8['dataset'] = 8
df_8 = df_8.loc[:,['dataset', 'text', 'logical_fallacies']]
df_8.head()

### 9

In [ ]:
df_9 = df_9[['text', 'fallacy']]
df_9 = df_9.rename(columns={'fallacy': 'logical_fallacies' })
df_9['dataset'] = 9
df_9 = df_9.loc[:,['dataset', 'text', 'logical_fallacies']]
df_9.head()

### lower case, underscore

In [157]:

def adjust(data):
    data['logical_fallacies']=data['logical_fallacies'].map(lambda x: x.lower().replace(' ', '_'))
    return data

df_1 = adjust(df_1)
df_2 = adjust(df_2)
df_3 = adjust(df_3)
df_4 = adjust(df_4)
df_5_1 = adjust(df_5_1)
df_5_2= adjust(df_5_2)
df_6 = adjust(df_6)
df_8 = adjust(df_8)
df_9 = adjust(df_9)

In [158]:
# dfList = [df_1, df_2, df_3, df_4 ,df_5_1, df_5_2, df_8, df_9]
# for d in dfList:
#     print (d.columns)


### concat

In [ ]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5_1, df_5_2, df_6, df_8, df_9])
df = df.reset_index(drop=True)
df

In [160]:
df = df[df['logical_fallacies']!='to_clean']

In [161]:
df = df[df['logical_fallacies']!='intentional']

In [ ]:
fall_list = df['logical_fallacies'].unique().tolist()
fall_list.sort()
fall_list

In [163]:
labels ={
    'none_of_the_above': 'none',
    '_ad_hominem': 'ad_hominem',
    '_hasty_generalization':'hasty_generalization',
    '_appeal_to_ridicule': 'appeal_to_ridicule',
    '_false_dilemma': 'false_dilemma',
    '_appeal_to_fear':'appeal_to_fear',
    'no_fallacy': 'none',
    'straw_man': 'strawman',
    'appeal_to_fear': 'appeal_to_emotion',
    'appeal_to_anger': 'appeal_to_emotion',
    'appeal_to_pity': 'appeal_to_emotion',
    'appeal_to_positive_emotion': 'appeal_to_emotion',
    'appeal_to_(false)_authority' : 'appeal_to_authority',
    'false_authority': 'appeal_to_authority',
    'hasty_generalization': 'faulty_generalization'
}

df= df.replace(labels)

In [ ]:
sns.countplot(data = df, y ='logical_fallacies')
plt.show()

In [ ]:
df.groupby('logical_fallacies')['logical_fallacies'].value_counts().sort_values(ascending=False)

## Drop missing values

In [166]:
df = df.dropna(subset=['text'])

## Clean text

In [ ]:
df.head()

In [ ]:
df.sort_values(by='text')

In [169]:
import re
#https://docs.python.org/3/library/re.html

#administration \' s
# \n

def clean_text(text):
    # Remove POST
    #text = re.sub(r'^.*?POST:', '', text, flags=re.DOTALL)
    # Remove any remaining "POST:" occurrences
    #text = re.sub(r'POST:', '', text)
    # Remove '\r\n' and extra whitespace
    #text = re.sub(r'\r\n', ' ', text)
    # Remove '\n'
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\r', '', text)
    # Remove " in beginning of text
    text = re.sub(r'^"', '', text)
    # Remove " at end of text
    text = re.sub(r'"$', '', text)
     # Remove ' in beginning of text
    text = re.sub(r"^'", '', text)
    # Remove ' at end of text
    text = re.sub(r"'$", '', text)
    # Remove (number or text) at beginning of text
    text = re.sub(r"^\([^)]*\)", '', text)
    # Remove \x80\x9d
    text = re.sub(r'\x80\x9d', '', text)
     # Remove : in beginning of text
    text = re.sub(r"^:", '', text)
    # Remove · in beginning of text
    text = re.sub(r"^•", '', text)
    # Remove ” in beginning of text
    text = re.sub(r"^”", '', text)
    # Remove @user
    text = re.sub(r'@user', '', text)
    # Remove [main_tweet], [user104337]
    text = re.sub(r"\[main_tweet\]", '', text)
    text = re.sub(r"\[user\d+\]", '', text)
     # Remove punctuation
    text = re.sub(r'\W', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

df['text'] = df['text'].apply(clean_text)

In [ ]:
df.loc[1641, "text"]

In [ ]:
txt = "hallo \r chocolatecake"

#Check if the string starts with 'hello':
#x = re.sub(r"\[user\d+\]", '', txt)
x = re.sub(r'\r', '', txt)
# x = re.sub(r"^\[[^)]*\]", '', txt)
print(x)


## Duplicates

In [ ]:
df.head()

In [ ]:
df_duplicated = df[df.duplicated(["text", "logical_fallacies"], keep=False)==True]
df_duplicated = df_duplicated.sort_values(by='text')
df_duplicated.head(20)

In [174]:
df = df.drop_duplicates(subset=['text', 'logical_fallacies'], keep='first')

In [ ]:
df.duplicated(['text']).sum()

In [ ]:
df.duplicated(['text', 'logical_fallacies']).sum()

In [ ]:
df['dataset'].value_counts().sort_index()

## Select fallacies

In [ ]:
df.groupby('logical_fallacies')['logical_fallacies'].value_counts().sort_values(ascending=False)

In [ ]:

most_freq_fallacies = df.groupby('logical_fallacies')['logical_fallacies'].value_counts().sort_values(ascending=False).index[:6]
most_freq_fallacies


## Save dataset

In [ ]:
df = df[df['logical_fallacies'].isin(most_freq_fallacies)]
df.info()

## Fine-Cleaning Dataset

In [181]:
df_clean2 = df.copy()

In [ ]:
df_clean2["logical_fallacies"].value_counts()

In [183]:
# Remove the following words and clean extra spaces
words_to_remove = ['uspoli', 'cdnpoli', 'pnpcbc', 'UKpolitics', 'auspol', 'polcan', 'eupol', 'nzpol', 'ctvpp']
pattern = r'\b(' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

df_clean2["text"] = df_clean2["text"].str.replace(pattern, '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()


In [184]:
# Look at one word texts and drop rows that do not make sense, e.g., testtesttesttest or are redundant
df_clean2 = df_clean2.drop(index=17908) #tessttetttetete	
df_clean2 = df_clean2.drop(index=18030) # tesforArgumentforAr
df_clean2 = df_clean2.drop(index=17573) # duplicate: yes
df_clean2 = df_clean2.drop(index=18337) # duplicate: no


In [185]:
pd.set_option("display.max_rows", None)
df_filtered = df_clean2[df_clean2['text'].str.split().str.len() == 1]

### Remove duplicated text

In [ ]:
# Look at duplicate text
df_duplicated_text = df_clean2[df_clean2.duplicated(["text"], keep=False)==True]
df_duplicated_text.shape

In [ ]:
# Drop duplicates
df_clean2 = df_clean2.drop_duplicates(subset=['text'], keep=False)
df_clean2.shape

### Save dataframe without duplicates

In [189]:
df_clean2.to_csv("../data/data.csv", index=False)